In [1]:
# Save with default filename
!wget https://github.com/arnabalumnus/Zebra/raw/main/python/DeviceAbuseDataFall.zip

--2021-06-08 10:50:13--  https://github.com/arnabalumnus/Zebra/raw/main/python/DeviceAbuseDataFall.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/arnabalumnus/Zebra/main/python/DeviceAbuseDataFall.zip [following]
--2021-06-08 10:50:13--  https://raw.githubusercontent.com/arnabalumnus/Zebra/main/python/DeviceAbuseDataFall.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1494369 (1.4M) [application/zip]
Saving to: ‘DeviceAbuseDataFall.zip.1’

DeviceAbuseDataFall 100%[===================>]   1.42M  --.-KB/s    in 0.05s   

2021-06-08 10:50:14 (26.9 MB/s) - ‘DeviceAbuseDataFall.zip.1’ saved 

In [2]:
# Unzip file
import zipfile
with zipfile.ZipFile('DeviceAbuseDataFall.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [3]:
import pandas as pd
import re
import os

In [4]:
def preProcess(rowCount):
    trainingDataFolder = "DeviceAbuseDataFall/"

    dirPath = os.path.join(os.getcwd(), trainingDataFolder)
    files = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]

    fileList = []
    finalDF = pd.DataFrame()
    
    for file in files:
        # Importing the dataset
        orgData = pd.read_csv(dirPath + file)
        data = pd.DataFrame(orgData.iloc[:, 1:4])

        print('Current File :', file, '# Rowcount :', data.shape[0])       
        if data.shape[0] > rowCount:
            fileList.append(file)
            start = 0
            end = rowCount
           
            count = 0
            for i in range(start, end):
                temp = pd.DataFrame(data.iloc[i:i+1, 0:3])
                temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
                temp = temp.reset_index(drop=True)
               
                if count == 0:
                    tempDF = temp
                    count = 1
                else:
                    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
                   
            finalDF = finalDF.append(tempDF)
        elif (data.shape[0] >= (rowCount / 2)):
            fileList.append(file)
            diff = rowCount - data.shape[0]
            start = 0
            end = rowCount
            count = 0
            countDown = rowCount - data.shape[0]

            for i in range(start, end):
                if (countDown > 0):
                    temp = pd.DataFrame(data.iloc[0:1, 0:3])
                    countDown -= 1
                else:
                    temp = pd.DataFrame(data.iloc[(i - diff):(i + 1 - diff), 0:3])

                temp.columns = ['X'+str(i), 'Y'+str(i), 'Z'+str(i)]
                temp = temp.reset_index(drop=True)
               
                if count == 0:
                    tempDF = temp
                    count = 1
                else:
                    tempDF = pd.concat([tempDF.reset_index(drop=True), temp], axis=1)
                   
            finalDF = finalDF.append(tempDF)
        else:
            print("No. of rows is less than", (rowCount / 2))
           
    fileList = [i.split('_', 1)[0] for i in fileList]
    fileList = [re.sub("\d+", "", i) for i in fileList]
    actualOutput = pd.DataFrame([fileName[:4] for fileName in fileList])
    actualOutput.columns = ["actualOutput"]
    
    finalDF = finalDF.reset_index(drop=True)
    finalDF = pd.concat([finalDF, actualOutput], axis=1)
    
    return finalDF

In [5]:
finalDF = preProcess(200)

# Clear logs
from IPython.display import clear_output
clear_output()

In [6]:
X_train = finalDF.iloc[:, 0:len(finalDF.columns)-1].values
y_train = finalDF.iloc[:, len(finalDF.columns)-1].values
   

In [7]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
svm_classifier = SVC(kernel = 'linear', random_state = 0)
svm_classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [8]:
import joblib
joblib.dump(svm_classifier, 'prefallClassifier_svm.pkl', compress=9) 

['prefallClassifier_svm.pkl']

In [9]:
# Fitting RF to the Training set
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
rf_classifier.fit(X_train, y_train)

joblib.dump(rf_classifier, 'prefallClassifier_rf.pkl', compress=9)

['prefallClassifier_rf.pkl']

# Error fix
Next block will have some error.
* To fix the error at **line-no 60** in `Porter.py` file 
* replace existing line with this below line   
```sklearn_ver = '0.22.2'.split('.')```
* After fix restart    
Goto `Runtime` -> `Restart and run all`


In [10]:
!pip install sklearn_porter

import sklearn_porter

porter=sklearn_porter.Porter(svm_classifier,language='java')
java_code=porter.export()
print(java_code)
f=open('MLPClassifier.java','w')
f.write(java_code)
f.close()



class SVC {

    private enum Kernel { LINEAR, POLY, RBF, SIGMOID }

    private int nClasses;
    private int nRows;
    private int[] classes;
    private double[][] vectors;
    private double[][] coefficients;
    private double[] intercepts;
    private int[] weights;
    private Kernel kernel;
    private double gamma;
    private double coef0;
    private double degree;

    public SVC (int nClasses, int nRows, double[][] vectors, double[][] coefficients, double[] intercepts, int[] weights, String kernel, double gamma, double coef0, double degree) {
        this.nClasses = nClasses;
        this.classes = new int[nClasses];
        for (int i = 0; i < nClasses; i++) {
            this.classes[i] = i;
        }
        this.nRows = nRows;

        this.vectors = vectors;
        this.coefficients = coefficients;
        this.intercepts = intercepts;
        this.weights = weights;

        this.kernel = Kernel.valueOf(kernel.toUpperCase());
        this.gamma = gamma;
        thi

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.weight_boosting module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. Th